In [15]:
import google.generativeai as genai
import os
import typing_extensions as typing
import json
from datasets import load_dataset

In [16]:
dataset = load_dataset("wikimedia/wikipedia", "20231101.en")['train']

In [17]:
class QAPairs(typing.TypedDict):
    question : str
    answer: str
    wrong_answers : list[str]

In [18]:
genai.configure(api_key='')

model = genai.GenerativeModel('gemini-1.5-flash')

few_shot = """
Base on the text, generate as many question answer and wrong answers pairs as possible but in Khmer. In each response, the question and answer are mandatory, the wrong answers are not.

Example:

Text:

The tomb of Philippe Pot is a life-sized funerary monument, now on display in the Louvre, Paris. It was commissioned by the military leader and diplomat Philippe Pot around the year 1480, to be used for his burial at the chapel of Saint-Jean-Baptiste in Cîteaux Abbey, Dijon, France. His effigy shows him recumbent on a slab, his hands raised in prayer, and wearing armour and a heraldic tunic. The eight mourners (pleurants) are dressed in black hoods, and act as pallbearers carrying him towards his grave. Pot commissioned the tomb when he was around 52 years old, 13 years before his death in 1493. The detailed inscriptions written on the sides of the slab emphasise his achievements and social standing.

Data generated:

Question 1: អ្នកណាជាអ្នកគាំទ្រនៃសិលាចារឹក Philippe Pot?
Answer 1: ក្រុមនិយាយទោស ៨ នាក់ដែលស្លៀកពាក់ខោមួកពណ៌ខ្មៅ

Wrong answer 1.1: ឈ្មោះចម្លាក់របស់ Louvre
Wrong answer 1.2: ព្រះសង្ឃនៅ Abbey Citeaux

Question 2: តើអ្នកចាំកំព្រាអ្នកស្លាប់នៅពេលណានៅក្នុងការគាំទ្រដល់ Philippe Pot?
Answer 2: ក្នុងឆ្នាំ ១៤៨០

Wrong answer 2.1: ក្នុងឆ្នាំ ១៤៩៣
Wrong answer 2.2: ក្នុងឆ្នាំ ១៤៥០

---------------------------------------------------------------------------------------------------------------------------

Text:



"""

In [19]:
def generate_questions_answers(text_chunk):
    
    response = model.generate_content(
        few_shot + text_chunk,
        generation_config=genai.GenerationConfig(
            response_mime_type="application/json", response_schema=list[QAPairs]
        ),
    )  
    try:
        response_text = response.text
        
        try:
            json_data = json.loads(response_text)
            return json_data
        except json.JSONDecodeError as e :
            print(e)
            return None
        
    except ValueError as e:
        print(e)
        return None
    
    

In [20]:

def process_text(text, i, chunk_size=4000):
    text_chunk = text[0 : chunk_size ]
    response = generate_questions_answers(text_chunk)
    
    if response is None: return
    
    with open('responses_0/' + str(i) + '.json', 'w', encoding='utf-8') as f:
        json.dump(response, f, indent=2, ensure_ascii=False)
    
    

In [ ]:
for i, data in enumerate(dataset):
    print(i)
    if i <= 740: continue
    process_text(data['text'], i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27